In [1]:
import tqdm
import pandas as pd
import numpy as np
import os

In [2]:
def paragraph_splited(doc,doc_id):
    para_id = 0
    splited = []
    data_frame = []
    for idx,d in enumerate(doc):
        for d_p in d.split('\n'):
            if d_p != '':
                data_frame.append([doc_id[idx],d_p])
                splited.append(d_p)
        para_id+=1
    data_frame = pd.DataFrame(data_frame ,columns =['para_id', 'para'])
    return splited,data_frame

# Cross-lingual

In [25]:
mode = 'train'
q_lan = ['en','ar','de','es','hi','vi','zh','ru','bn','fi','ja','ko','te','fr','th','el','ro','tr','zh']
# q_lan = ['ru']

corpus = 'XORQA'

path = f'data_preprocess/{corpus}/{mode}/'
top_start = 1  
top_end = 3

df_question = {}
df_paragraph = {}
df_doc = {}
df_doc_m = {}
lan_now = []
for lan in q_lan:
    try:
        question_temp = pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_question_en-{lan}.csv')
        doc_temp = pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_doc_en-{lan}.csv')
        question_temp = question_temp.dropna()

        df_question.update({
            f'en-{lan}': question_temp
        })
        df_doc_m.update({
            f'en-{lan}': doc_temp
        })
        
#         df_paragraph.update({
#             f'en-{lan}':pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_para_en-{lan}.csv')
#         })
        
        lan_now.append(lan)
    except:
        pass
    
q_lan = lan_now


In [17]:
context_mode = 'para'
for idx,lan in enumerate(q_lan[:1]):
    print(f'Lan:{idx+1}/{len(q_lan)}')
    question_id = df_question[f'en-{lan}']['doc_id'].to_list()
    questions = df_question[f'en-{lan}']['question'].to_list()
    
    doc_context_id = df_doc_m[f'en-{lan}']['doc_id'].to_list()
    doc_context_raw = df_doc_m[f'en-{lan}']['doc'].to_list()
    
    para_split,para_df = paragraph_splited(doc_context_raw,doc_context_id)
    
    df_merged = pd.merge(df_question[f'en-{lan}'],para_df,left_on='doc_id',right_on='para_id')
    try:
        df_merged = df_merged.drop(['paragraph_id','para_id'], axis=1)
    except:
        df_merged = df_merged.drop(['para_id'], axis=1)
    df_merged.dropna(inplace=True)
    df_merged.drop_duplicates(inplace=True)
    df_merged.to_csv(f'{path}/triplet/triplet_en-{lan}_en_top0-0_{context_mode}.csv',index=False)

Lan:1/6


In [5]:
context_mode = 'para_aligned'
for idx,lan in enumerate(q_lan[:]):
    print(f'Lan:{idx+1}/{len(q_lan)}')
    df_merged = pd.merge(df_question[f'en-{lan}'],df_paragraph[f'en-{lan}'],left_on='paragraph_id',right_on='para_id')
    df_merged = df_merged.drop(['paragraph_id','doc_id_y','para_id','title'], axis=1)
    df_merged.columns = ['doc_id','question','para']
    df_merged.to_csv(f'{path}/triplet/triplet_en-{lan}_en_top0-0_{context_mode}_mlqa.csv',index=False)
    

Lan:1/8
Lan:2/8
Lan:3/8
Lan:4/8
Lan:5/8
Lan:6/8
Lan:7/8
Lan:8/8


In [26]:
context_mode = 'doc'
for idx,lan in enumerate(q_lan[:]):
    print(f'Lan:{idx+1}/{len(q_lan)}')
    question_id = df_question[f'en-{lan}']['doc_id'].to_list()
    questions = df_question[f'en-{lan}']['question'].to_list()

    df_merged = pd.merge(df_question[f'en-{lan}'],df_doc_m[f'en-{lan}'],left_on='doc_id',right_on='doc_id')
    try:
        df_merged = df_merged.drop(['paragraph_id','title'], axis=1)
    except:
        df_merged = df_merged.drop(['title'], axis=1)
    df_merged.columns = ['doc_id','question','para']  
    df_merged.to_csv(f'{path}/triplet/triplet_en-{lan}_top0-0_{context_mode}.csv',index=False)

Lan:1/6
Lan:2/6
Lan:3/6
Lan:4/6
Lan:5/6
Lan:6/6
